In [81]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import re  # 用于文本预处理
from collections import Counter  # 统计词汇
import math

In [82]:
# 文件路径（替换为你的实际路径）
file_path = r"D:\朱思潮\编程\学习编程\PYTHON\学习python\数据分析与机器学习\基本算法语法\机器学习\RNN与transformer\秒速5厘米.txt"

# 读取文件内容
with open(file_path, "r", encoding="utf-8") as f:  # "r"表示读取，encoding指定为utf-8处理中文

    raw_text = f.read()


In [83]:
# ----------------------
# 1. 数据预处理
# ----------------------
def preprocess_text(text):
    """预处理文本：清洗、分词、构建词汇表"""
    # 简单清洗（可根据需求扩展，如去除特殊符号、小写化等）
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9\s]', '', text)  # 保留中文、英文、数字和空格
    # 分词（中文可用jieba，这里简化为按字符分割）
    chars = list(text)  # 按字符级处理，适合中文
    # 构建词汇表
    vocab = sorted(list(set(chars)))
    char_to_idx = {u:i for i, u in enumerate(vocab)}
    idx_to_char = {i:u for i, u in enumerate(vocab)}
    vocab_size = len(vocab)
    # 将文本转为索引序列
    text_indices = [char_to_idx[c] for c in chars]
    return text_indices, char_to_idx, idx_to_char, vocab_size


In [84]:

text_indices, char_to_idx, idx_to_char, vocab_size = preprocess_text(raw_text)


In [85]:
# 构建训练数据（输入序列->目标序列）
seq_length = 100  # 输入序列长度（影响模型对上下文的记忆范围）
dataX, dataY = [], []
for i in range(len(text_indices) - seq_length):
    seq_in = text_indices[i:i+seq_length]
    seq_out = text_indices[i+1:i+seq_length+1]  # 目标是输入的下一个字符
    dataX.append(seq_in)
    dataY.append(seq_out)

# 转换为Tensor
dataX = torch.tensor(dataX, dtype=torch.long)
dataY = torch.tensor(dataY, dtype=torch.long)



In [86]:
# ----------------------
# 2. Transformer模型定义（适配文本生成）
# ----------------------
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)  # 形状为(1, max_len, d_model)
        self.register_buffer('pe', pe)  # 注册为不参与训练的缓冲区

    def forward(self, x):
        # x: (batch_size, seq_length, d_model)
        # 修正：直接添加位置编码，无需额外unsqueeze
        x = x + self.pe[:, :x.size(1), :]  # 自动广播到batch维度
        return x

# ----------------------
# 修正后的Transformer模型（使用编码器作为解码器）
# ----------------------
class TransformerTextGenerator(nn.Module):
    def __init__(self, vocab_size, d_model=256, nhead=4, num_layers=2, dim_feedforward=512):
        super().__init__()
        self.d_model = d_model
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        
        # 使用Transformer编码器作为解码器（纯解码器架构）
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.fc_out = nn.Linear(d_model, vocab_size)

    def forward(self, src, src_mask=None):
        # 嵌入 + 位置编码
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        
        # 通过Transformer（应用因果掩码）
        output = self.transformer(src, mask=src_mask)
        
        # 输出预测
        return self.fc_out(output)

# ----------------------
# 生成掩码函数（保持不变）
# ----------------------
def generate_square_subsequent_mask(sz):
    mask = torch.triu(torch.ones(sz, sz), diagonal=1).bool()
    return mask.float().masked_fill(mask, float('-inf'))


In [87]:
# ----------------------
# 修正后的训练循环
# ----------------------
# 超参数
batch_size = 16
n_epochs = 10
lr = 0.001

# 初始化模型
model = TransformerTextGenerator(
    vocab_size=vocab_size,  # 确保vocab_size已定义
    d_model=256,
    nhead=4,
    num_layers=2
)

# 设备配置（关键修复！）
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
# 训练循环
for epoch in range(n_epochs):
    model.train()
    total_loss = 0
    permutation = torch.randperm(len(dataX))
    
    for i in range(0, len(dataX), batch_size):
        # 取批次数据
        indices = permutation[i:i+batch_size]
        
        # 确保数据在正确设备上
        batch_src = dataX[indices].to(device)  # 输入序列
        batch_tgt_y = dataY[indices].to(device)  # 目标序列
        
        # 生成掩码（确保在相同设备上）
        seq_len = batch_src.size(1)
        src_mask = generate_square_subsequent_mask(seq_len).to(device)
        
        # 前向传播
        output = model(batch_src, src_mask=src_mask)
        
        # 计算损失（确保维度匹配）
        # output形状: (batch, seq_len, vocab_size)
        # 需要转换为: (batch, vocab_size, seq_len)
        loss = criterion(output.transpose(1, 2), batch_tgt_y)
        
        # 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # 打印每轮平均损失
    avg_loss = total_loss / (len(dataX) // batch_size)
    print(f"Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}")

Epoch 1/10, Loss: 2.3239
Epoch 2/10, Loss: 1.3833


In [ ]:
# ----------------------
# 4. 文本生成函数
# ----------------------
def generate_text(model, start_str, max_length=500, temperature=0.7):
    """基于起始字符串生成文本"""
    model.eval()
    generated = list(start_str)
    # 起始字符串转为索引
    input_seq = [char_to_idx[c] for c in start_str]
    input_tensor = torch.tensor(input_seq).unsqueeze(0)  # 形状：(1, seq_len)
    
    with torch.no_grad():
        for _ in range(max_length):
            # 生成当前序列的掩码
            seq_len = input_tensor.size(1)
            tgt_mask = generate_square_subsequent_mask(seq_len).to(input_tensor.device)
            
            # 预测下一个字符（取输出序列的最后一个位置）
            output = model(input_tensor, tgt_mask=tgt_mask)
            next_token_logits = output[0, -1, :]  # 取第一个样本的最后一个位置
            
            # 用temperature调整概率分布（控制随机性）
            next_token_probs = F.softmax(next_token_logits / temperature, dim=-1)
            next_token_idx = torch.multinomial(next_token_probs, num_samples=1).item()
            
            # 更新生成结果和输入序列
            generated_char = idx_to_char[next_token_idx]  # 假设idx2char已定义
            generated.append(generated_char)
            input_tensor = torch.cat([input_tensor, torch.tensor([[next_token_idx]])], dim=1)
    
    return ''.join(generated)


# 示例：生成文本
start_text = "明月几时有，"
generated = generate_text(model, start_text, max_length=300)
print("生成结果：\n", generated)